In [1]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [2]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [3]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [4]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [5]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [6]:
print("emb matrix shape", emb_matrix.shape)
print("vocab len", len(vocab))


emb matrix shape (10001, 300)
vocab len 10000


In [10]:
imp.reload(util)
seqlen = 128
X, Xu = util.load_data(path_train, vocab, pad=seqlen, numfiles=100)

  0%|          | 73/22860 [00:00<00:31, 724.30it/s]


In [25]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, Concatenate
from keras import Model, Input
import tensorflow as tf

In [59]:
A = np.zeros((3, 10, 5))

for i in range(3):
    A[i,:,:] = np.reshape(np.array(range(50)), (10,5)) + i*100

#print(A)
    #A
#var = K.variable

indices = np.array([
    [[0,0], [0,5]],
    [[1,1], [1,6]],
    [[2,2], [2,7]]
], dtype="int32")

#indices = np.array([0,5
#], dtype="int32")

v = K.backend.variable(A)
k = K.backend.variable(indices, dtype="int32")

pp = tf.shape(v)[0]
print("pp",pp)

g = tf.gather_nd(A, indices)

q = K.backend.eval(g)
print(q.shape)
print(q)

pp Tensor("strided_slice:0", shape=(), dtype=int32)
(3, 2, 5)
[[[  0.   1.   2.   3.   4.]
  [ 25.  26.  27.  28.  29.]]

 [[105. 106. 107. 108. 109.]
  [130. 131. 132. 133. 134.]]

 [[210. 211. 212. 213. 214.]
  [235. 236. 237. 238. 239.]]]


In [88]:
q = np.array([
    [0,1,2,0],
    [1,2,3,1],
    [2,3,4,2]
])

qv = K.backend.variable(q, dtype="int32")

def mygather(e):
    batch, indices = e
    print(type(batch))
    print(batch.shape)
    print(indices.shape)
    #i2 = tf.expand_dims(indices)
    #print(i2.shape)
    return (tf.gather(batch, indices),0)

r = tf.map_fn(mygather, (v, qv))

print(K.backend.eval(r[0]))

<class 'tensorflow.python.framework.ops.Tensor'>
(10, 5)
(4,)
[[[  0.   1.   2.   3.   4.]
  [  5.   6.   7.   8.   9.]
  [ 10.  11.  12.  13.  14.]
  [  0.   1.   2.   3.   4.]]

 [[105. 106. 107. 108. 109.]
  [110. 111. 112. 113. 114.]
  [115. 116. 117. 118. 119.]
  [105. 106. 107. 108. 109.]]

 [[210. 211. 212. 213. 214.]
  [215. 216. 217. 218. 219.]
  [220. 221. 222. 223. 224.]
  [210. 211. 212. 213. 214.]]]


In [91]:
def sampling_layer(x):
    data, sample_indices = x
    def gather_per_batch(e):
        batch, indices = e
        return (tf.gather(batch, indices),0)
    return tf.map_fn(gather_per_batch, (data, sample_indices))[0]


In [92]:
z = sampling_layer([v, qv])

K.backend.eval(z)

array([[[  0.,   1.,   2.,   3.,   4.],
        [  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.],
        [  0.,   1.,   2.,   3.,   4.]],

       [[105., 106., 107., 108., 109.],
        [110., 111., 112., 113., 114.],
        [115., 116., 117., 118., 119.],
        [105., 106., 107., 108., 109.]],

       [[210., 211., 212., 213., 214.],
        [215., 216., 217., 218., 219.],
        [220., 221., 222., 223., 224.],
        [210., 211., 212., 213., 214.]]], dtype=float32)

In [54]:
from keras.layers import Reshape, Embedding, CuDNNLSTM, BatchNormalization, Dense, Concatenate
from keras import Model, Input

def make_model(*, seqlen, sample_size, emb_matrix):    
    dim = emb_matrix.shape[1] + 1
    
    input_x = Input((seqlen,), dtype="int32", name="input_x")
    input_xu = Input((seqlen,), dtype="float32", name="input_xu")
    input_sample_indices = Input((sample_size,), dtype="int32", name="input_sample_indices")

    resh_xu  = Reshape((seqlen,1), name="resh_xu")(input_xu)

    emb_layer = Embedding(*emb_matrix.shape, input_length=seqlen, 
                          trainable=False, weights=[emb_matrix], 
                          name="embedding")
    emb_x = emb_layer(input_x)
    concat_x = Concatenate(name="concat_x")([emb_x, resh_xu])
        
    yhat = CuDNNLSTM(128, return_sequences=True, name='rnn1')(concat_x)
    yhat = BatchNormalization()(yhat)
    yhat = CuDNNLSTM(128, return_sequences=False, name='rnn2')(yhat)

    yhat = BatchNormalization()(yhat)
    yhat = Dense(300, activation="relu")(yhat)
    yhat = BatchNormalization()(yhat)
    # len(vocab)+2 is because the zeroth word is for padding and last word is for "unknown"
    yhat = Dense(len(vocab)+2, activation="linear")(yhat)
    
    print(yhat.shape)

In [55]:
make_model(seqlen=seqlen, sample_size=5, emb_matrix=emb_matrix)

(?, 10002)


In [37]:


yhat = K.layers.CuDNNLSTM(128, return_sequences=True, name='rnn1')(concat_x)
yhat = K.layers.BatchNormalization()(yhat)
yhat = K.layers.CuDNNLSTM(128, return_sequences=False, name='rnn2')(yhat)

yhat = K.layers.BatchNormalization()(yhat)
yhat = K.layers.Dense(300, activation="relu")(yhat)
yhat = K.layers.BatchNormalization()(yhat)
dense = K.layers.Dense(len(vocab)+2, activation="softmax")(dense)

#dense = K.layers.BatchNormalization()(lstm)
#lstm = K.layers.CuDNNLSTM(256, return_sequences=True)(lstm)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)
#dense = K.layers.BatchNormalization()(dense)
#dense = K.layers.Dense(dim, activation="sigmoid")(dense)



In [38]:
model = K.Model(inputs=[input_x, input_xu],
               outputs=[dense])

In [39]:
Yu.shape

(152720, 128)

In [57]:
batch_size=16

import time

def data_generator(X, Xu, Y, Yu, batch_size=batch_size, vocab_size=10000):
    batches = int(math.floor(X.shape[0]/batch_size))
    while True:
        for i in range(batches):
            t0 = time.time()
            start, end = i*batch_size, (i+1)*batch_size
            bX  = X[start:end]
            bXu = Xu[start:end]
            Yc = K.utils.to_categorical(Y[start:end], num_classes=vocab_size)
            bY = np.concatenate((Yc,
                            np.expand_dims(Yu[start:end], axis=2)), axis=2)
            t1 = time.time() - t0
            #print("time", t1)
            yield [[bX, bXu], [bY]]
        
datagen = data_generator(X, Xu, Y, Yu, vocab_size=len(vocab)+1)

b = next(datagen)

In [58]:
b
zY = np.argmax(b[1][0], axis=2)
#print(b[0][0][0])
#print(zY[0])

print(b[0][0].size, b[0][1].size, b[1][0].size)

2048 2048 20484096


In [61]:
opt = K.optimizers.Adam(lr=0.1)
model.compile(opt, loss='categorical_crossentropy')

In [ ]:
import math

numbatches = int(math.floor(X.shape[0]/batch_size))
model.fit_generator(datagen, steps_per_epoch=numbatches, epochs=5000,
                   use_multiprocessing=False)


Epoch 1/5000
37/37 [==============================] - 8s 228ms/step - loss: 6.6803
Epoch 2/5000
37/37 [==============================] - 8s 228ms/step - loss: 6.5376
Epoch 3/5000
 3/37 [=>............................] - ETA: 7s - loss: 6.5944

In [17]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="categorical_crossentropy")

In [19]:
i = 7
gen = X[i].tolist()
genu = Xu[i].tolist()

tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
results = []

UNK_IDX = len(words) + 1

iterations = 32
for j in range(iterations):
    tX[0,:] = np.array(gen[-seqlen:], "int32")
    tXu[0,:] = np.array(genu[-seqlen:], "float32")
    
    #print(tX)
    z = predictor.predict([tX, tXu])
    scores=z[0, -1]
    idx = UNK_IDX
    while idx == UNK_IDX:
        idx = np.random.choice(range(len(vocab)+2), p=scores)

    gen.append(idx)
    genu.append(0.0)
    results.append(words[idx])
    sys.stdout.write(results[-1] + " ")
    sys.stdout.flush()
#    print("%s ", results[-1])

IndexError: list index out of range

1.3547268e-16 0.93638355


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'